In [1]:
import cv2
import random
import numpy as np
import albumentations as A

from glob import glob
from torch.utils.data import Dataset
from albumentations.pytorch import ToTensorV2

In [2]:
data_dir = "/home/pervinco/Datasets/BKAI_IGH_NeoPolyp"

In [ ]:
def get_transform(is_train, img_size):
    if is_train:
        transform = A.Compose([
            A.Resize(img_size, img_size, p=1, always_apply=True),
            A.HorizontalFlip(p=0.5),
            A.VerticalFlip(p=0.5),
            
            A.RandomGamma (gamma_limit=(70, 130), eps=None, always_apply=False, p=0.2),
            A.RGBShift(p=0.3, r_shift_limit=10, g_shift_limit=10, b_shift_limit=10),
            
            A.OneOf([A.Blur(), A.GaussianBlur(), A.GlassBlur(), A.MotionBlur(), A.GaussNoise(), A.Sharpen(), A.MedianBlur(), A.MultiplicativeNoise()]),

            A.CoarseDropout(p=0.2, max_height=35, max_width=35, fill_value=0, mask_fill_value=0),
            A.RandomSnow(snow_point_lower=0.1, snow_point_upper=0.15, brightness_coeff=1.5, p=0.09),
            A.RandomShadow(p=0.1),
            A.ShiftScaleRotate(p=0.45, border_mode=cv2.BORDER_CONSTANT, shift_limit=0.15, scale_limit=0.15, rotate_limit=180),
            A.RandomCrop(img_size, img_size)])
    else:
        transform = A.Compose([A.Resize(img_size, img_size, p=1, always_apply=True)])

    return transform

In [ ]:
def mask_encoding(mask):  
    hsv_mask = cv2.cvtColor(mask, cv2.COLOR_BGR2HSV)
    
    # lower boundary RED color range values; Hue (0 - 10)
    lower1 = np.array([0, 100, 20])
    upper1 = np.array([10, 255, 255])
    
    # upper boundary RED color range values; Hue (160 - 180)
    lower2 = np.array([160,100,20])
    upper2 = np.array([179,255,255])

    lower_mask = cv2.inRange(hsv_mask, lower1, upper1)
    upper_mask = cv2.inRange(hsv_mask, lower2, upper2)
    red_mask = lower_mask + upper_mask
    red_mask[red_mask != 0] = 2

    # boundary RED color range values; Hue (36 - 70)
    green_mask = cv2.inRange(hsv_mask, (36, 25, 25), (70, 255,255))
    green_mask[green_mask != 0] = 1
    full_mask = cv2.bitwise_or(red_mask, green_mask)
    full_mask = full_mask.astype(np.uint8)

    return full_mask


def mask_decoding(pred_mask):
    decoded_mask = np.zeros((pred_mask.shape[0], pred_mask.shape[1], 3), dtype=np.uint8)
    decoded_mask[pred_mask == 0] = [0, 0, 0]
    decoded_mask[pred_mask == 1] = [0, 255, 0] ## Green
    decoded_mask[pred_mask == 2] = [255, 0, 0] ## Red
    
    return decoded_mask

In [3]:
class BKAIDataset(Dataset):
    def __init__(self, data_dir, img_size):
        self.img_size = img_size
        self.data_dir = data_dir
        self.image_dir = f"{self.data_dir}/train"
        self.mask_dir = f"{self.data_dir}/train_gt"
        self.transform = get_transform(is_train=True, img_size=self.img_size)

        self.image_files = sorted(glob(f"{self.image_dir}/*.jpeg"))
        self.mask_files = sorted(glob(f"{self.mask_dir}/*.jpeg"))
        
        self.total_files = list(zip(self.image_files, self.mask_files))

    def __len__(self):
        return len(self.total_files)
    
    def __getitem__(self, idx):
        image_path, mask_path = self.total_files[idx]

        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

        mask = cv2.imread(mask_path)
        
        transformed = self.transform(image=image, mask=mask)
        image, mask = transformed["image"], transformed["mask"]

        encoded_mask = mask_encoding(mask)

        return image, encoded_mask

In [4]:
dataset = BKAIDataset(data_dir, img_size=512)

1000
